## loading patient names

This file is written by Chitipotu Kushwanth

In [ ]:
import numpy as np

def load_patient_ids_from_file(input_file):
    """Loads patient IDs from a specified input file."""
    with open(input_file, 'r') as f:
        patient_ids = [line.strip() for line in f.readlines()]
    return patient_ids

# Dataset splitting

In [ ]:
patient_ids = load_patient_ids_from_file(r"D:\BraTS2021_Training_Datapatient_ids.txt")
patient_ids = patient_ids[1:]
# Example patient splits (use your actual splits)
np.random.seed(47)
indices = np.arange(0, len(patient_ids))
np.random.shuffle(indices)

train_ids = [patient_ids[i] for i in indices[:int(0.7 * len(indices))]]
test_ids = [patient_ids[i] for i in indices[int(0.7 * len(indices)):int(0.8 * len(indices))]]
validation_ids = [patient_ids[i] for i in indices[int(0.8 * len(indices)):]]


# Using Resnet(not working)

In [ ]:
import os
import torch
import torch.nn as nn
import nibabel as nib
import numpy as np
import torch.nn.functional as F

# Define ResidualBlock and Downsample3D (from the provided code)
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv3d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm3d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv3d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm3d(out_channels)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual
        out = self.relu(out)
        return out

class Downsample3D(nn.Module):
    def __init__(self, in_channels=1, base_channels=1):
        super(Downsample3D, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv3d(in_channels, base_channels, kernel_size=2, stride=2, padding=0),
            nn.BatchNorm3d(base_channels),
            nn.ReLU(inplace=True),

            ResidualBlock(base_channels, base_channels),
            nn.Conv3d(base_channels, base_channels, kernel_size=2, stride=2, padding=0),
            nn.BatchNorm3d(base_channels),
            nn.ReLU(inplace=True),

            ResidualBlock(base_channels, base_channels),
            nn.Conv3d(base_channels, base_channels, kernel_size=2, stride=1, padding=1),
            nn.BatchNorm3d(base_channels),
            nn.ReLU(inplace=True),

            ResidualBlock(base_channels, base_channels),
            nn.Conv3d(base_channels, base_channels, kernel_size=2, stride=1, padding=1),
            nn.BatchNorm3d(base_channels),
            nn.ReLU(inplace=True),

            ResidualBlock(base_channels, base_channels),
            nn.Conv3d(base_channels, base_channels, kernel_size=2, stride=1, padding=1),
            nn.BatchNorm3d(base_channels),
            nn.ReLU(inplace=True),

            ResidualBlock(base_channels, base_channels),
            nn.Conv3d(base_channels, base_channels, kernel_size=2, stride=1, padding=1),
            nn.BatchNorm3d(base_channels),
            ResidualBlock(base_channels, base_channels)
        )

    def forward(self, x):

        out = self.encoder(x)
        out = F.interpolate(out, size=(64, 64, 64), mode='trilinear', align_corners=True)
        return out

# Initialize the model
model = Downsample3D(in_channels=1)  # Input is single-channel images
model.eval()  # Set the model to evaluation mode

# Paths and directories
input_dir = r"D:\BraTS2021_Training_Data"  # Directory containing patient folders
output_dir = r"C:\Processed_Data"  # Base directory to save processed files
os.makedirs(output_dir, exist_ok=True)

# Function to process patient files
def process_patient_files(patient_ids, set_name):
    set_output_dir = os.path.join(output_dir, set_name)
    os.makedirs(set_output_dir, exist_ok=True)

    for patient_id in patient_ids:
        patient_dir = os.path.join(input_dir, patient_id)
        output_patient_dir = os.path.join(set_output_dir, patient_id)
        os.makedirs(output_patient_dir, exist_ok=True)

        for file_type in ["flair", "t1", "t1ce", "t2", "seg"]:
            input_file = os.path.join(patient_dir, f"{patient_id}_{file_type}.nii")
            output_file = os.path.join(output_patient_dir, f"{patient_id}_{file_type}.pt")

            if not os.path.exists(input_file):
                print(f"File not found: {input_file}")
                continue

            # Process segmentation files (copy as-is)
            # if file_type == "seg":
            #     os.system(f'copy "{input_file}" "{output_file}"')
            #     print(f"Copied segmentation file: {input_file} -> {output_file}")
            #     continue

            # Process other files (reduce dimensions)
            img = nib.load(input_file)
            img_data = img.get_fdata()  # Extract image data as a numpy array
            img_tensor = torch.tensor(img_data, dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # Add batch and channel dims

            # Reduce dimensions using the model
            with torch.no_grad():
                reduced_tensor = model(img_tensor)
            reduced_tensor = torch.tensor(reduced_tensor, dtype=torch.float32).squeeze(0).squeeze(0)
            print(f"Processed tensor shape: {reduced_tensor.shape}")

            # Save reduced tensor
            torch.save(reduced_tensor, output_file)
            print(f"Processed and saved: {input_file} -> {output_file}")

# Example function to load patient IDs from a text file



# Load patient IDs from file

# Process each set
process_patient_files(train_ids, "train")
# process_patient_files(test_ids, "test")
# process_patient_files(validation_ids, "validation")


C:\Users\intel5\AppData\Local\Temp\ipykernel_5900\1882942318.py:110: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  reduced_tensor = torch.tensor(reduced_tensor, dtype=torch.float32).squeeze(0).squeeze(0)


Processed tensor shape: torch.Size([64, 64, 64])
Processed and saved: D:\BraTS2021_Training_Data\BraTS2021_01083\BraTS2021_01083_flair.nii -> C:\Processed_Data\train\BraTS2021_01083\BraTS2021_01083_flair.pt
Processed tensor shape: torch.Size([64, 64, 64])
Processed and saved: D:\BraTS2021_Training_Data\BraTS2021_01083\BraTS2021_01083_t1.nii -> C:\Processed_Data\train\BraTS2021_01083\BraTS2021_01083_t1.pt
Processed tensor shape: torch.Size([64, 64, 64])
Processed and saved: D:\BraTS2021_Training_Data\BraTS2021_01083\BraTS2021_01083_t1ce.nii -> C:\Processed_Data\train\BraTS2021_01083\BraTS2021_01083_t1ce.pt
Processed tensor shape: torch.Size([64, 64, 64])
Processed and saved: D:\BraTS2021_Training_Data\BraTS2021_01083\BraTS2021_01083_t2.nii -> C:\Processed_Data\train\BraTS2021_01083\BraTS2021_01083_t2.pt
Processed tensor shape: torch.Size([64, 64, 64])
Processed and saved: D:\BraTS2021_Training_Data\BraTS2021_01083\BraTS2021_01083_seg.nii -> C:\Processed_Data\train\BraTS2021_01083\BraTS2

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import os
from torchvision import transforms

class BRATS2021Dataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (string): Root directory where the modality .pt files and segmentation .pt files are stored.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.root_dir = root_dir
        self.patients = sorted(os.listdir(root_dir))  # List of patient folders
        self.transform = transform

    def __len__(self):
        return len(self.patients)

    def __getitem__(self, idx):
        patient_dir = os.path.join(self.root_dir, self.patients[idx])

        # Define the modality filenames
        modality_1_path = os.path.join(patient_dir, f"{self.patients[idx]}_flair.pt")
        modality_2_path = os.path.join(patient_dir, f"{self.patients[idx]}_t1.pt")
        modality_3_path = os.path.join(patient_dir, f"{self.patients[idx]}_t1ce.pt")
        modality_4_path = os.path.join(patient_dir, f"{self.patients[idx]}_t2.pt")

        # Load modalities from .pt files
        modality_1 = torch.load(modality_1_path, weights_only=False)
        modality_2 = torch.load(modality_2_path, weights_only=False)
        modality_3 = torch.load(modality_3_path, weights_only=False)
        modality_4 = torch.load(modality_4_path, weights_only=False)


        # Stack the modalities into a single tensor (e.g., shape: [4, H, W, D])
        modalities = torch.stack([modality_1, modality_2, modality_3, modality_4], dim=0)

        # Define the segmentation path and load the segmentation tensor from .pt file
        segmentation_path = os.path.join(patient_dir, f"{self.patients[idx]}_seg.pt")

        if not os.path.exists(segmentation_path):
            print(f"Segmentation file not found: {segmentation_path}")
            return None  # Or handle the error in a way that makes sense for your case

        try:
            segmentation = torch.load(segmentation_path, weights_only=False)
        except Exception as e:
            print(f"Error loading segmentation file: {e}")
  # assuming it's a tensor saved with torch.save()


        # Apply transformations if any
        if self.transform:
            modalities = self.transform(modalities)
            segmentation = self.transform(segmentation)


        modalities = modalities.permute(1, 2, 3, 0)

        return modalities,segmentation



# Define any transformations (e.g., resizing, normalization, etc.)
transform = transforms.Compose([
    transforms.Normalize(mean=[0.5, 0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5, 0.5])  # Normalize modalities
])




In [ ]:
train_dir = r'C:/Processed_Data/train'
train_dataset = BRATS2021Dataset(root_dir=train_dir)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

test_dir = r'C:/Processed_Data/test'
test_dataset = BRATS2021Dataset(root_dir=test_dir)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True)

validation_dir = r'C:/Processed_Data/validation'  # Change to your dataset path
validation_dataset = BRATS2021Dataset(root_dir=validation_dir)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True)

In [ ]:
i=0
for data,target in train_dataloader:
    i+=1
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219


# Using Monai Transforms

In [ ]:
import numpy as np

def load_patient_ids_from_file(input_file):
    """Loads patient IDs from a specified input file."""
    with open(input_file, 'r') as f:
        patient_ids = [line.strip() for line in f.readlines()]
    return patient_ids

# Example usage
input_file = r"D:\BraTS2021_Training_Datapatient_ids.txt"
patient_ids = load_patient_ids_from_file(input_file)



In [ ]:
print(patient_ids)

['.DS_Store', 'BraTS2021_00000', 'BraTS2021_00002', 'BraTS2021_00003', 'BraTS2021_00005', 'BraTS2021_00006', 'BraTS2021_00008', 'BraTS2021_00009', 'BraTS2021_00011', 'BraTS2021_00012', 'BraTS2021_00014', 'BraTS2021_00016', 'BraTS2021_00017', 'BraTS2021_00018', 'BraTS2021_00019', 'BraTS2021_00020', 'BraTS2021_00021', 'BraTS2021_00022', 'BraTS2021_00024', 'BraTS2021_00025', 'BraTS2021_00026', 'BraTS2021_00028', 'BraTS2021_00030', 'BraTS2021_00031', 'BraTS2021_00032', 'BraTS2021_00033', 'BraTS2021_00035', 'BraTS2021_00036', 'BraTS2021_00043', 'BraTS2021_00044', 'BraTS2021_00045', 'BraTS2021_00046', 'BraTS2021_00048', 'BraTS2021_00049', 'BraTS2021_00051', 'BraTS2021_00052', 'BraTS2021_00053', 'BraTS2021_00054', 'BraTS2021_00056', 'BraTS2021_00058', 'BraTS2021_00059', 'BraTS2021_00060', 'BraTS2021_00061', 'BraTS2021_00062', 'BraTS2021_00063', 'BraTS2021_00064', 'BraTS2021_00066', 'BraTS2021_00068', 'BraTS2021_00070', 'BraTS2021_00071', 'BraTS2021_00072', 'BraTS2021_00074', 'BraTS2021_00077'

In [ ]:
import os
import glob
import numpy as np
import nibabel as nib
from monai.transforms import (
    LoadImaged,
    Spacingd,
    Resized,
    EnsureTyped,
    Compose
)
from monai.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Define paths
base_dir = r"D:\BraTS2021_Training_Data"
output_dir = r"C:\Processed_BraTS2021"
os.makedirs(output_dir, exist_ok=True)

# Get patient folders
patient_folders = sorted(glob.glob(os.path.join(base_dir, "BraTS2021_*")))

# Create data list for MONAI
data_list = []
for folder in patient_folders:
    patient_id = os.path.basename(folder)  # Extract patient ID
    data_list.append({
        "patient_id": patient_id,
        "flair": os.path.join(folder, f"{patient_id}_flair.nii"),
        "t1": os.path.join(folder, f"{patient_id}_t1.nii"),
        "t1ce": os.path.join(folder, f"{patient_id}_t1ce.nii"),
        "t2": os.path.join(folder, f"{patient_id}_t2.nii"),
        "seg": os.path.join(folder, f"{patient_id}_seg.nii")
    })

# Split data into train, val, and test sets
train_data, test_data = train_test_split(data_list, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

# Define preprocessing transforms
transforms = Compose([
    LoadImaged(keys=["flair", "t1", "t1ce", "t2", "seg"]),
    Spacingd(keys=["flair", "t1", "t1ce", "t2", "seg"], pixdim=(1.0, 1.0, 1.0), mode=("bilinear", "bilinear", "bilinear", "bilinear", "nearest")),
    Resized(keys=["flair", "t1", "t1ce", "t2", "seg"], spatial_size=(64, 64, 64), mode=("area", "area", "area", "area", "nearest")),
    EnsureTyped(keys=["flair", "t1", "t1ce", "t2", "seg"])
])

# Create datasets
train_ds = Dataset(data=train_data, transform=transforms)
val_ds = Dataset(data=val_data, transform=transforms)
test_ds = Dataset(data=test_data, transform=transforms)

# Create data loaders
batch_size = 4  # Process one patient at a time
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

# Function to save processed data
def save_processed_data(loader, split_name):
    split_dir = os.path.join(output_dir, split_name)
    os.makedirs(split_dir, exist_ok=True)

    for batch in loader:
        patient_id = batch["patient_id"][0]  # Get patient ID
        patient_dir = os.path.join(split_dir, patient_id)
        os.makedirs(patient_dir, exist_ok=True)  # Create folder for patient

        for modality in ["flair", "t1", "t1ce", "t2", "seg"]:
            image = batch[modality][0].numpy()  # Extract the image
            output_path = os.path.join(patient_dir, f"{patient_id}_{modality}.nii")
            nib.save(nib.Nifti1Image(image, affine=np.eye(4)), output_path)
            print(f"Saved {modality} file for {patient_id}: {output_path}")

# Save train, val, and test data
save_processed_data(train_loader, "train")
save_processed_data(val_loader, "val")
save_processed_data(test_loader, "test")


Saved flair file for BraTS2021_01381: C:\Processed_BraTS2021\train\BraTS2021_01381\BraTS2021_01381_flair.nii
Saved t1 file for BraTS2021_01381: C:\Processed_BraTS2021\train\BraTS2021_01381\BraTS2021_01381_t1.nii
Saved t1ce file for BraTS2021_01381: C:\Processed_BraTS2021\train\BraTS2021_01381\BraTS2021_01381_t1ce.nii
Saved t2 file for BraTS2021_01381: C:\Processed_BraTS2021\train\BraTS2021_01381\BraTS2021_01381_t2.nii
Saved seg file for BraTS2021_01381: C:\Processed_BraTS2021\train\BraTS2021_01381\BraTS2021_01381_seg.nii
Saved flair file for BraTS2021_01163: C:\Processed_BraTS2021\train\BraTS2021_01163\BraTS2021_01163_flair.nii
Saved t1 file for BraTS2021_01163: C:\Processed_BraTS2021\train\BraTS2021_01163\BraTS2021_01163_t1.nii
Saved t1ce file for BraTS2021_01163: C:\Processed_BraTS2021\train\BraTS2021_01163\BraTS2021_01163_t1ce.nii
Saved t2 file for BraTS2021_01163: C:\Processed_BraTS2021\train\BraTS2021_01163\BraTS2021_01163_t2.nii
Saved seg file for BraTS2021_01163: C:\Processed_Br

OSError: [Errno 28] No space left on device

In [ ]:
import os
import glob
import numpy as np
import nibabel as nib
from monai.transforms import (
    LoadImaged,
    Spacingd,
    Resized,
    EnsureTyped,
    Compose,
    EnsureChannelFirstd,
)
from monai.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Define paths
base_dir = r"D:\BraTS2021_Training_Data"
output_dir = r"C:\Users\intel5\Downloads\Processed_BraTS2021"
os.makedirs(output_dir, exist_ok=True)

# Get patient folders
patient_folders = sorted(glob.glob(os.path.join(base_dir, "BraTS2021_*")))

# Create data list for MONAI
data_list = []
for folder in patient_folders:
    patient_id = os.path.basename(folder)  # Extract patient ID
    data_list.append({
        "patient_id": patient_id,
        "flair": os.path.join(folder, f"{patient_id}_flair.nii"),
        "t1": os.path.join(folder, f"{patient_id}_t1.nii"),
        "t1ce": os.path.join(folder, f"{patient_id}_t1ce.nii"),
        "t2": os.path.join(folder, f"{patient_id}_t2.nii"),
        "seg": os.path.join(folder, f"{patient_id}_seg.nii")
    })

# Split data into train, val, and test sets
train_data, test_data = train_test_split(data_list, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

# Define preprocessing transforms
transforms = Compose([
    LoadImaged(keys=["flair", "t1", "t1ce", "t2", "seg"], ensure_channel_first=True),
    Spacingd(keys=["flair", "t1", "t1ce", "t2", "seg"], pixdim=(1.0, 1.0, 1.0), mode=("bilinear", "bilinear", "bilinear", "bilinear", "nearest")),
    Resized(keys=["flair", "t1", "t1ce", "t2", "seg"], spatial_size=(64, 64, 64), mode=("area", "area", "area", "area", "nearest")),
    EnsureTyped(keys=["flair", "t1", "t1ce", "t2", "seg"]),
])

# Create datasets
train_ds = Dataset(data=train_data, transform=transforms)
val_ds = Dataset(data=val_data, transform=transforms)
test_ds = Dataset(data=test_data, transform=transforms)

# Create data loaders
batch_size = 1  # Process one patient at a time
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

# Function to save processed data
def save_processed_data(loader, split_name):
    split_dir = os.path.join(output_dir, split_name)
    os.makedirs(split_dir, exist_ok=True)

    for batch in loader:
        for i in range(len(batch["patient_id"])):
            patient_id = batch["patient_id"][i]  # Get patient ID
            patient_dir = os.path.join(split_dir, patient_id)
            os.makedirs(patient_dir, exist_ok=True)  # Create folder for patient

            for modality in ["flair", "t1", "t1ce", "t2", "seg"]:
                # Retrieve original file path
                original_file = data_list[i][modality]
                original_img = nib.load(original_file)
                original_dtype = original_img.get_data_dtype()  # Get original data type
                original_affine = original_img.affine  # Get original affine

                # Convert processed image back to original data type
                image = batch[modality][i].numpy().astype(original_dtype)

                # Save processed image with the original affine
                output_path = os.path.join(patient_dir, f"{patient_id}_{modality}.nii")
                nib.save(nib.Nifti1Image(image, affine=original_affine), output_path)
                print(f"Saved {modality} file for {patient_id}: {output_path}")

# Save train, val, and test data
save_processed_data(train_loader, "train")
save_processed_data(val_loader, "val")
save_processed_data(test_loader, "test")


Saved flair file for BraTS2021_00674: C:\Users\intel5\Downloads\Processed_BraTS2021\train\BraTS2021_00674\BraTS2021_00674_flair.nii
Saved t1 file for BraTS2021_00674: C:\Users\intel5\Downloads\Processed_BraTS2021\train\BraTS2021_00674\BraTS2021_00674_t1.nii
Saved t1ce file for BraTS2021_00674: C:\Users\intel5\Downloads\Processed_BraTS2021\train\BraTS2021_00674\BraTS2021_00674_t1ce.nii
Saved t2 file for BraTS2021_00674: C:\Users\intel5\Downloads\Processed_BraTS2021\train\BraTS2021_00674\BraTS2021_00674_t2.nii
Saved seg file for BraTS2021_00674: C:\Users\intel5\Downloads\Processed_BraTS2021\train\BraTS2021_00674\BraTS2021_00674_seg.nii
Saved flair file for BraTS2021_01293: C:\Users\intel5\Downloads\Processed_BraTS2021\train\BraTS2021_01293\BraTS2021_01293_flair.nii
Saved t1 file for BraTS2021_01293: C:\Users\intel5\Downloads\Processed_BraTS2021\train\BraTS2021_01293\BraTS2021_01293_t1.nii
Saved t1ce file for BraTS2021_01293: C:\Users\intel5\Downloads\Processed_BraTS2021\train\BraTS2021_0